**Using EfficientB7 as feature extractor for task 1**


In [6]:
import warnings
import pickle
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')

batch_size = 16

# Load EfficientNet-B7 model
model = models.efficientnet_b7(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL image
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert PIL image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = self.transform(img)
        return img

class LWPClassifier:
    def __init__(self, num_classes=2):
        self.num_classes = num_classes
        self.prototypes = None  # Prototypes will be initialized during training

    def fit(self, X, y):
        # Initialize prototypes as the mean of features for each class
        self.prototypes = np.array([X[y == c].mean(axis=0) for c in range(self.num_classes)])

    def predict(self, X):
        # Compute distances to prototypes
        distances = np.linalg.norm(X[:, np.newaxis] - self.prototypes, axis=2)  # Shape: [num_samples, num_classes]
        return np.argmin(distances, axis=1)  # Assign to the nearest prototype

def get_features(images, transform):
    dataset = UnlabeledDataset(images, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    feature_extractor.eval()
    features = []

    with torch.no_grad():
        for batch_images in loader:
            batch_images = batch_images.to(device)
            batch_features = feature_extractor(batch_images)  # Shape: [batch_size, 2560, 7, 7]
            batch_features = batch_features.mean([2, 3])  # Global Average Pooling: [batch_size, 2560]
            features.append(batch_features.cpu())

    # Combine features into a single tensor
    features = torch.cat(features)
    return features

all_eval_features = []
all_eval_labels = []
acc_matrix = np.zeros((10, 10))
prev_feature = []
prev_labels = []
data_paths = [f"/content/drive/MyDrive/train_data/{i}_train_data.tar.pth" for i in range(1, 11)]
heldout_paths = [f"/content/drive/MyDrive/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]

for loop in range(10):
    data = torch.load(data_paths[loop])
    heldout = torch.load(heldout_paths[loop])

    train_images = data['data']
    train_labels = data['targets'] if loop == 0 else prev_labels
    eval_images = heldout['data']
    eval_labels = heldout['targets']

    feature_extractor = feature_extractor.to(device)

    train_features = get_features(train_images, transform) if loop == 0 else prev_feature
    print(f"{loop} Done training features")

    f = LWPClassifier(num_classes=10)
    f.fit(train_features, train_labels)

    if loop == 9:
        with open("lwp_classifier.pkl", "wb") as file:
            pickle.dump(f, file)

    eval_features = get_features(eval_images, transform)
    print(f"{loop} Done eval features")
    all_eval_features.append(eval_features)
    all_eval_labels.append(eval_labels)

    accuracies = []
    for j in range(loop + 1):
        preds = f.predict(all_eval_features[j])
        acc = accuracy_score(all_eval_labels[j], preds)
        accuracies.append(acc)
        acc_matrix[loop][j] = acc

    pred_labels = f.predict(eval_features)
    acc = accuracy_score(eval_labels, pred_labels)
    accuracies.append(acc)
    acc_matrix[loop][loop] = acc
    print(f"{loop} Test Accuracy: {accuracies}")

    if loop != 9:
        next_data = torch.load(data_paths[loop + 1])
        next_images = next_data['data']
        next_features = get_features(next_images, transform)
        print(f"{loop} Done next features")
        prev_feature = next_features
        prev_labels = f.predict(next_features)

print(acc_matrix)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0 Done training features
0 Done eval features
0 Test Accuracy: [0.8504, 0.8504]
0 Done next features
1 Done training features
1 Done eval features
1 Test Accuracy: [0.8264, 0.8564, 0.8564]
1 Done next features
2 Done training features
2 Done eval features
2 Test Accuracy: [0.8188, 0.8484, 0.8332, 0.8332]
2 Done next features
3 Done training features
3 Done eval features
3 Test Accuracy: [0.818, 0.8484, 0.8308, 0.8388, 0.8388]
3 Done next features
4 Done training features
4 Done eval features
4 Test Accuracy: [0.8144, 0.8416, 0.826, 0.8348, 0.8428, 0.8428]
4 Done next features
5 Done training features
5 Done eval features
5 Test Accuracy: [0.8104, 0.8368, 0.822, 0.8288, 0.8312, 0.8208, 0.8208]
5 Done next features
6 Done training features
6 Done eval features
6 Test Accuracy: [0.814, 0.8368, 0.8192, 0.832, 0.8344, 0.8208, 0.8252, 0.8252]
6 Done next features
7

**Using EfficientB7 as feature extractor for task 2**

In [7]:
import pickle

# Load the classifier from the file
with open("lwp_classifier.pkl", "rb") as file:
    f = pickle.load(file)

import warnings
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive')
batch_size = 16

# Load EfficientNet-B7 as the feature extractor
model = models.efficientnet_b7(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the classification head
feature_extractor.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL image
    transforms.Resize((224, 224)),  # Resize images to 224x224 (EfficientNet-B7 default input size)
    transforms.ToTensor(),  # Convert PIL image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, images, transform):  # Corrected __init__
        self.images = images
        self.transform = transform

    def __len__(self):  # Corrected __len__
        return len(self.images)

    def __getitem__(self, idx):  # Corrected __getitem__
        img = self.images[idx]
        img = self.transform(img)
        return img

def get_features(images, transform):
    dataset = UnlabeledDataset(images, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    feature_extractor.eval()
    features = []

    with torch.no_grad():
        for batch_images in loader:
            batch_images = batch_images.to(device)
            batch_features = feature_extractor(batch_images)  # Extract features
            batch_features = batch_features.mean([2, 3])  # Global Average Pooling
            features.append(batch_features.cpu())

    # Combine features into a single tensor
    features = torch.cat(features)
    return features

# Initialize variables
all_eval_features = []
all_eval_labels = []
acc_matrix = np.zeros((10, 10))
data_paths = [f"/content/drive/MyDrive/train_data02/{i}_train_data.tar.pth" for i in range(1, 11)]
heldout_paths = [f"/content/drive/MyDrive/eval_data02/{i}_eval_data.tar.pth" for i in range(1, 11)]

# Main training and evaluation loop
for loop in range(10):
    data = torch.load(data_paths[loop])
    heldout = torch.load(heldout_paths[loop])

    train_images = data['data']
    eval_images = heldout['data']
    eval_labels = heldout['targets']

    feature_extractor = feature_extractor.to(device)

    # Extract training features
    train_features = get_features(train_images, transform)
    print(f"{loop} Done training features")
    train_labels = f.predict(train_features)

    # Fit the classifier
    f.fit(train_features, train_labels)

    # Extract evaluation features
    eval_features = get_features(eval_images, transform)
    print(f"{loop} Done eval features")
    all_eval_features.append(eval_features)
    all_eval_labels.append(eval_labels)

    # Compute accuracies
    accuracies = []
    for j in range(loop):
        preds = f.predict(all_eval_features[j])
        acc = accuracy_score(all_eval_labels[j], preds)
        accuracies.append(acc)
        acc_matrix[loop][j] = acc

    pred_labels = f.predict(eval_features)
    acc = accuracy_score(eval_labels, pred_labels)
    accuracies.append(acc)
    acc_matrix[loop][loop] = acc
    print(f"{loop} Test Accuracy: {accuracies}")

# Print the accuracy matrix
print(acc_matrix)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0 Done training features
0 Done eval features
0 Test Accuracy: [0.6348]
1 Done training features
1 Done eval features
1 Test Accuracy: [0.5524, 0.3552]
2 Done training features
2 Done eval features
2 Test Accuracy: [0.596, 0.3508, 0.6244]
3 Done training features
3 Done eval features
3 Test Accuracy: [0.608, 0.3468, 0.6384, 0.6984]
4 Done training features
4 Done eval features
4 Test Accuracy: [0.636, 0.3568, 0.6496, 0.7216, 0.7648]
5 Done training features
5 Done eval features
5 Test Accuracy: [0.6088, 0.3396, 0.634, 0.6952, 0.7364, 0.6028]
6 Done training features
6 Done eval features
6 Test Accuracy: [0.6152, 0.3312, 0.6336, 0.6956, 0.7432, 0.5996, 0.6816]
7 Done training features
7 Done eval features
7 Test Accuracy: [0.6024, 0.3376, 0.6288, 0.6924, 0.728, 0.5944, 0.6676, 0.5992]
8 Done training features
8 Done eval features
8 Test Accuracy: [0.5944, 0.34

**Using ConvNext as feature extractor for task 1**


In [1]:
import warnings
import pickle
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import timm  # For ConvNeXt models
import numpy as np
from sklearn.metrics import accuracy_score
from google.colab import drive

drive.mount('/content/drive')

batch_size = 16

# Load ConvNeXt model
model = timm.create_model('convnext_small', pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])  # Remove classifier head
feature_extractor.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL image
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert PIL image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = self.transform(img)
        return img

class LWPClassifier:
    def __init__(self, num_classes=2):
        self.num_classes = num_classes
        self.prototypes = None  # Prototypes will be initialized during training

    def fit(self, X, y):
        # Initialize prototypes as the mean of features for each class
        self.prototypes = np.array([X[y == c].mean(axis=0) for c in range(self.num_classes)])

    def predict(self, X):
        # Compute distances to prototypes
        distances = np.linalg.norm(X[:, np.newaxis] - self.prototypes, axis=2)  # Shape: [num_samples, num_classes]
        return np.argmin(distances, axis=1)  # Assign to the nearest prototype

def get_features(images, transform):
    dataset = UnlabeledDataset(images, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    feature_extractor.eval()
    features = []

    with torch.no_grad():
        for batch_images in loader:
            batch_images = batch_images.to(device)
            batch_features = feature_extractor(batch_images)  # Shape: [batch_size, feature_dim, H, W]
            batch_features = batch_features.mean([2, 3])  # Global Average Pooling
            features.append(batch_features.cpu())

    # Combine features into a single tensor
    features = torch.cat(features)
    return features

all_eval_features = []
all_eval_labels = []
acc_matrix = np.zeros((10, 10))
prev_feature = []
prev_labels = []
data_paths = [f"/content/drive/MyDrive/train_data/{i}_train_data.tar.pth" for i in range(1, 11)]
heldout_paths = [f"/content/drive/MyDrive/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)]

for loop in range(10):
    data = torch.load(data_paths[loop])
    heldout = torch.load(heldout_paths[loop])

    train_images = data['data']
    train_labels = data['targets'] if loop == 0 else prev_labels
    eval_images = heldout['data']
    eval_labels = heldout['targets']

    feature_extractor = feature_extractor.to(device)

    train_features = get_features(train_images, transform) if loop == 0 else prev_feature
    print(f"{loop} Done training features")

    f = LWPClassifier(num_classes=10)
    f.fit(train_features, train_labels)

    if loop == 9:
        with open("lwp_classifier.pkl", "wb") as file:
            pickle.dump(f, file)

    eval_features = get_features(eval_images, transform)
    print(f"{loop} Done eval features")
    all_eval_features.append(eval_features)
    all_eval_labels.append(eval_labels)

    accuracies = []
    for j in range(loop + 1):
        preds = f.predict(all_eval_features[j])
        acc = accuracy_score(all_eval_labels[j], preds)
        accuracies.append(acc)
        acc_matrix[loop][j] = acc

    pred_labels = f.predict(eval_features)
    acc = accuracy_score(eval_labels, pred_labels)
    accuracies.append(acc)
    acc_matrix[loop][loop] = acc
    print(f"{loop} Test Accuracy: {accuracies}")

    if loop != 9:
        next_data = torch.load(data_paths[loop + 1])
        next_images = next_data['data']
        next_features = get_features(next_images, transform)
        print(f"{loop} Done next features")
        prev_feature = next_features
        prev_labels = f.predict(next_features)

print(acc_matrix)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

0 Done training features
0 Done eval features
0 Test Accuracy: [0.9316, 0.9316]
0 Done next features
1 Done training features
1 Done eval features
1 Test Accuracy: [0.9268, 0.93, 0.93]
1 Done next features
2 Done training features
2 Done eval features
2 Test Accuracy: [0.924, 0.9256, 0.9304, 0.9304]
2 Done next features
3 Done training features
3 Done eval features
3 Test Accuracy: [0.924, 0.9252, 0.932, 0.9312, 0.9312]
3 Done next features
4 Done training features
4 Done eval features
4 Test Accuracy: [0.9228, 0.9284, 0.934, 0.9312, 0.934, 0.934]
4 Done next features
5 Done training features
5 Done eval features
5 Test Accuracy: [0.9216, 0.9248, 0.9296, 0.9248, 0.928, 0.9292, 0.9292]
5 Done next features
6 Done training features
6 Done eval features
6 Test Accuracy: [0.9144, 0.9204, 0.9232, 0.9228, 0.9216, 0.9224, 0.932, 0.932]
6 Done next features
7 Done training features
7 Done eval features
7 Test Accuracy: [0.918, 0.9196, 0.9232, 0.9216, 0.9244, 0.926, 0.9312, 0.9208, 0.9208]
7 Do

**Using ConvNext as feature extractor for task 2**

In [2]:
import pickle

# Load the classifier from the file
with open("lwp_classifier.pkl", "rb") as file:
    f = pickle.load(file)

import warnings
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import timm  # For ConvNeXt models
import numpy as np
from sklearn.metrics import accuracy_score
from google.colab import drive

drive.mount('/content/drive')
batch_size = 16

# Load ConvNeXt-Large as the feature extractor
model = timm.create_model('convnext_small', pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the classification head
feature_extractor.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert tensor to PIL image
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ConvNeXt default input size)
    transforms.ToTensor(),  # Convert PIL image back to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img = self.transform(img)
        return img

def get_features(images, transform):
    dataset = UnlabeledDataset(images, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    feature_extractor.eval()
    features = []

    with torch.no_grad():
        for batch_images in loader:
            batch_images = batch_images.to(device)
            batch_features = feature_extractor(batch_images)  # Extract features
            batch_features = batch_features.mean([2, 3])  # Global Average Pooling
            features.append(batch_features.cpu())

    # Combine features into a single tensor
    features = torch.cat(features)
    return features

# Initialize variables
all_eval_features = []
all_eval_labels = []
acc_matrix = np.zeros((10, 10))
data_paths = [f"/content/drive/MyDrive/train_data02/{i}_train_data.tar.pth" for i in range(1, 11)]
heldout_paths = [f"/content/drive/MyDrive/eval_data02/{i}_eval_data.tar.pth" for i in range(1, 11)]

# Main training and evaluation loop
for loop in range(10):
    data = torch.load(data_paths[loop])
    heldout = torch.load(heldout_paths[loop])

    train_images = data['data']
    eval_images = heldout['data']
    eval_labels = heldout['targets']

    feature_extractor = feature_extractor.to(device)

    # Extract training features
    train_features = get_features(train_images, transform)
    print(f"{loop} Done training features")
    train_labels = f.predict(train_features)

    # Fit the classifier
    f.fit(train_features, train_labels)

    # Extract evaluation features
    eval_features = get_features(eval_images, transform)
    print(f"{loop} Done eval features")
    all_eval_features.append(eval_features)
    all_eval_labels.append(eval_labels)

    # Compute accuracies
    accuracies = []
    for j in range(loop):
        preds = f.predict(all_eval_features[j])
        acc = accuracy_score(all_eval_labels[j], preds)
        accuracies.append(acc)
        acc_matrix[loop][j] = acc

    pred_labels = f.predict(eval_features)
    acc = accuracy_score(eval_labels, pred_labels)
    accuracies.append(acc)
    acc_matrix[loop][loop] = acc
    print(f"{loop} Test Accuracy: {accuracies}")

# Print the accuracy matrix
print(acc_matrix)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0 Done training features
0 Done eval features
0 Test Accuracy: [0.7684]
1 Done training features
1 Done eval features
1 Test Accuracy: [0.7336, 0.6276]
2 Done training features
2 Done eval features
2 Test Accuracy: [0.7476, 0.6064, 0.8192]
3 Done training features
3 Done eval features
3 Test Accuracy: [0.7616, 0.6008, 0.8244, 0.9032]
4 Done training features
4 Done eval features
4 Test Accuracy: [0.7508, 0.5928, 0.8232, 0.9036, 0.9056]
5 Done training features
5 Done eval features
5 Test Accuracy: [0.7488, 0.5912, 0.8244, 0.8864, 0.8956, 0.7688]
6 Done training features
6 Done eval features
6 Test Accuracy: [0.7416, 0.588, 0.8124, 0.8836, 0.8872, 0.7708, 0.8264]
7 Done training features
7 Done eval features
7 Test Accuracy: [0.7464, 0.594, 0.8232, 0.88, 0.886, 0.7592, 0.8268, 0.7948]
8 Done training features
8 Done eval features
8 Test Accuracy: [0.7236, 0.57